 # Simulating CH4 with PennyLane 

## Building molecular Hamiltonian 

In [ ]:
from pennylane import numpy as np 
from pennylane import qchem 
import pennylane as qml 
import time 

# defining the molecule 

symbols = ["C", "H", "H", "H", "H"]
coordinates = 0.529*np.array([[0.0,0.0,0.0],[0.6276,0.6276,0.6276],[0.6276,-0.6276,-0.6276],[-0.6276,0.6276,-0.6276],[-0.6276,-0.6276,0.6276]])
# coordinates in atomic units 

In [ ]:
# defining molecule specificites for simplification after having studied it's MOs

charge = 0  # not an ion 
mult = 1 # initially, prone to change if Givens rotations 
active_electrons = 8 # not considering the 1s core electrons 
active_orbitals = 5 # neglecting core 1s and the 3 MOs with the highest energy doesn't change much with 6 active orbitals instead of 5 (tried out in case considered 1s coming from C, which is irrelevant)

H, qubits = qchem.molecular_hamiltonian(symbols,coordinates,charge=charge, mult = mult, active_electrons = active_electrons, active_orbitals = active_orbitals)


## Energy and ground state 

In [ ]:
electrons = 8
hf = qml.qchem.hf_state(electrons, qubits) # creating corresponding Hartree-Fock state
print(hf)

### Evaluating the relevant Givens rotations (i.e. electrons excitations)

In [ ]:
singles, doubles = qchem.excitations(active_electrons,qubits)
print("Total number of excitations = {}".format(len(singles)+len(doubles))) # yields 24 possible spin-preserving excitations

def circuit_1(params, excitations): 
    qml.BasisState(hf,wires=range(qubits)) # generating the corresponding Hartree-Fock state in the the VQC
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4: # meaning a double-excitation 
            qml.DoubleExcitation(params[i],wires=excitation) # where the excitation is being applied
        else: # meaning single-excitation 
            qml.SingleExcitation(params[i],wires=excitation) 
    return qml.expval(H)

""" 
Protocol : 

- Compute gradients for all double excitations.

- Select the double excitations with gradients larger than a pre-defined threshold.

- Perform VQE to obtain the optimized parameters for the selected double excitations.

- Repeat steps 1 and 2 for the single excitations.

- Perform the final VQE optimization with all the selected excitations.
"""

In [ ]:
# double excitations selection

dev = qml.device("default.qubit",wires=qubits)
cost_fn = qml.QNode(circuit_1,dev) # instead of the decorator, same thing otherwise

circuit_gradient = qml.grad(cost_fn,argnum=0) # returns the gradient as a callable function of (functions of) QNodes.

params = [0.0] * len(doubles) # parameter values to zero such that the gradients are computed with respect to the Hartree-Fock state.
grads = circuit_gradient(params,excitations=doubles) # OK 

#for i in range(len(doubles)):
    #print(f"Excitation : {doubles[i]}, Gradient : {grads[i]}")

In [ ]:
# defining the right threshold in order not to have to many Givens rotations 

doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-2]
# len(doubles_select), if 1.0e-3 takes all the excitations

In [ ]:
# finding the optimizing parameters for double excitations 

opt = qml.GradientDescentOptimizer(stepsize=0.5) # QNG doesn't seem to provide any speed-up
params_doubles = np.zeros(len(doubles_select),requires_grad = True) 

for n in range(20): # number of optimizations 
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select) 

In [ ]:
# single excitations selection (same idea but have to prior consider the previously selected double excitations)

def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf, wires=range(qubits))
    for i, gate in enumerate(gates_select): # applying the selected double excitations 
        qml.DoubleExcitation(params_select[i], wires=gate)       
    for i, gate in enumerate(excitations): # testing the single excitations 
        qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)

cost_fn = qml.QNode(circuit_2, dev)
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)

grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)

#for i in range(len(singles)):
    #print(f"Excitation : {singles[i]}, Gradient: {grads[i]}") # f format string

In [ ]:
# defining the right threshold in order not to have to many Givens rotations 

singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-4]
# len(singles_select)

In [ ]:
# applying all selected excitations and VQE to optimize the full quantum circuit 

# before, to speed things up, let's consider the sparsing (lot of zeroes) of the molecular Hamiltonian 

H_sparse = qml.utils.sparse_hamiltonian(H)

# now the VQE 

opt = qml.GradientDescentOptimizer(stepsize=0.5) # what about QNG ?

excitations = doubles_select + singles_select

params = np.zeros(len(excitations), requires_grad=True)

@qml.qnode(dev, diff_method="parameter-shift")
def circuit(params):
    qml.BasisState(hf, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        elif len(excitation) == 2:
            qml.SingleExcitation(params[i], wires=excitation)

    return qml.expval(qml.SparseHamiltonian(H_sparse, wires=range(qubits)))


""" for n in range(20):
    t1 = time.time()
    params, energy = opt.step_and_cost(circuit, params)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))"""

conv_tol = 1e-06
gd_cost = []
max_iterations = 500

for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(circuit, params)
    gd_cost.append(prev_energy)

    energy = circuit(params)
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print(
            "Iteration = {:},  Energy = {:.8f} Ha".format(n, energy)
        )

    if conv <= conv_tol:
        break
        

plt.style.use("seaborn")
plt.plot(np.array(gd_cost), "g", label="Gradient descent")
plt.ylabel("Energy")
plt.xlabel("Step")
plt.legend()
plt.show()

In [ ]:
# comparing to automatically found eigenvalues :
Hs = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H), wires=range(qubits))
qml.eigvals(Hs,k=5) # same order of magnitude !

# Geometric optimization of H20

### Building the parametrized electronic Hamiltonian

Geometry optimization for $H_2O$

In [ ]:
from pennylane import numpy as np
#1
symbols = ["H", "O", "H"]
x = np.array([-0.0399, -0.0038, 0.0, 1.5780, 0.8540, 0.0, 2.7909, -0.5159, 0.0], requires_grad=True)
# initial set of coordinates 

We build a parameterized electronic hamiltonian $H(x)$ using the Jordan-Wogner transformation
$$
H(x)=\sum_jh_j(x)\prod_i^Nσ(j)_i^{(j)}.
$$


In [ ]:
#2
import pennylane as qml

def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=1)[0]

### Variational quantum circuit 

Six qubits are required to describe all the molecular orbitals (see H2O MOs diagram).
To capture the effects of electronic correlations, we need to prepare the N-qubit system in a superposition of the Hartree-Fock state $|11111111000000⟩$ with other states that differ by a double- or single-excitation.
Then generate indices of the qubits involved in all possible single and double excitations. In addition, we need to apply an adaptative algorithm to eliminate some possibilities.

In [ ]:
#3
hf = qml.qchem.hf_state(electrons=8, orbitals=14)
print(hf)

In [ ]:
#4
num_wires = 14
dev = qml.device("default.qubit", wires=num_wires)


@qml.qnode(dev)
def circuit(params, obs, wires):
    qml.BasisState(hf, wires=wires)# to initialize the qubit register
    qml.DoubleExcitation(params[0], wires=[6, 7, 10, 11])
    qml.DoubleExcitation(params[1], wires=[4, 5, 8, 9])

    return qml.expval(obs)

This circuit prepares the state:
$$
|Ψ(θ1,θ2)⟩=cos(θ1)cos(θ2)|11111111000000⟩−cos(θ1)sin(θ2)|11111100001100⟩−sin(θ1)|11110011110000⟩
$$

### The cost function and the nuclear gradients
Defining the cost function
$$
g(θ,x)=⟨Ψ(θ)|H(x)|Ψ(θ)⟩
$$
The cost function here depends on both the circuit and the hamiltonian parameters

In [2]:
#5
def cost(params, x):
    hamiltonian = H(x)
    return circuit(params, obs=hamiltonian, wires=range(num_wires))

The nuclear gradients are evaluated by taking the expectation value of the gradient of the electronic Hamiltonian,
$$
∇_xg(θ,x)=⟨Ψ(θ)|∇_xH(x)|Ψ(θ)⟩.
$$

We use the finite_diff() function to compute the gradient of the Hamiltonian using a central-difference approximation.

In [ ]:
#6
def finite_diff(f, x, delta=0.01):
    """Compute the central-difference finite difference of a function"""
    gradient = []

    for i in range(len(x)):
        shift = np.zeros_like(x)
        shift[i] += 0.5 * delta
        res = (f(x + shift) - f(x - shift)) * delta**-1
        gradient.append(res)

    return gradient

Then, we evaluate the expectation value of the gradient components $\frac{∂H(x)}{∂x_i}$. This is implemented by the function grad_x:

In [ ]:
#7
def grad_x(params, x):
    grad_h = finite_diff(H, x)
    grad = [circuit(params, obs=obs, wires=range(num_wires)) for obs in grad_h]
    return np.array(grad)

### Optimization of the molecular geometry
we proceed to minimize our cost function to find the ground state equilibrium geometry.
the circuit parameters and the nuclear coordinates will be jointly optimized at each optimization step

In [ ]:
#8
# defining the clasical optimizers
opt_theta = qml.GradientDescentOptimizer(stepsize=0.4)
opt_x = qml.GradientDescentOptimizer(stepsize=0.8)

In [ ]:
#9
# initializing theta 1 and theta 2
theta = np.array([0.0,0.0], requires_grad=True)

The circuit parameters and the nuclear coordinates are optimized until the maximum component of the nuclear gradient $∇_xg(θ,x)$ is less than or equal to $10^{−5}$ Hartree/Bohr. Typically, this is the convergence criterion used for optimizing molecular geometries in quantum chemistry simulations.

In [ ]:
#10
from functools import partial
import time

# store the values of the cost function
energy = []

# store the values of the bond length
bond_length = []

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903


for n in range(4):
    start = time.time()
    # Optimize the circuit parameters
    theta.requires_grad = True
    x.requires_grad = False
    theta, _ = opt_theta.step(cost, theta, x)

    # Optimize the nuclear coordinates
    x.requires_grad = True
    theta.requires_grad = False
    _, x = opt_x.step(cost, theta, x, grad_fn=grad_x)

    energy.append(cost(theta, x))
    bond_length.append(np.linalg.norm(x[0:3] - x[3:6]) * bohr_angs)
    
    print(f"Step = {n},  E = {energy[-1]:.8f} Ha,  bond length = {bond_length[-1]:.5f} A")
    end = time.time()
    print(f"Time {n}=",end - start)
    
    # Check maximum component of the nuclear gradient
    if np.max(grad_x(theta, x)) <= 1e-05:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" "Ground-state equilibrium geometry")
print("%s %4s %8s %8s" % ("symbol", "x", "y", "z"))
for i, atom in enumerate(symbols):
    print(f"  {atom}    {x[3 * i]:.4f}   {x[3 * i + 1]:.4f}   {x[3 * i + 2]:.4f}")



#4400 sec
#2750 sec

# print(x)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams["figure.figsize"] = (7,5)
mpl.rcParams.update({'font.size': 16})
plt.figure()
plt.plot(x[0],x[1], 'o',label='H')
plt.plot(x[3],x[4], 'o',label='O')
plt.plot(x[6],x[7], 'o',label='H')

LigneX1=[x[0],x[3]]
LigneY1=[x[1],x[4]]
LigneX2=[x[3],x[6]]
LigneY2=[x[4],x[7]]

plt.plot(LigneX1,LigneY1,'--',color='b')
plt.plot(LigneX2,LigneY2,'--',color='b')

plt.grid()
plt.xlabel('X axis in Angstrom')
plt.ylabel('Y axis in Angstrom')

plt.legend()
plt.tight_layout()
plt.savefig("H20_geo")
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = (7,5)
mpl.rcParams.update({'font.size': 16})

Number=[1,2,3,4]
fig, ax1 = plt.subplots()
ax1.plot(Number,energy, 'o' ,color='blue')
ax2 = ax1.twinx()
ax2.plot(Number,bond_length, 'o' ,color='red')

ax1.set_xlabel('Number of iteration')
ax1.set_ylabel('Ground state energy given in Ha')
ax2.set_ylabel('Bond length in Angstrom')
ax1.grid()
fig.tight_layout()
plt.savefig('Energy_bond')
plt.show()
